In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Embedding, LSTM, SimpleRNN,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.stem import WordNetLemmatizer

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
df1=pd.read_csv("/content/drive/MyDrive/SemEval2016-Task6-raw-annotations-stance.csv")
#df=df1.ffill()
df=df1
df

,Worker ID,Instance ID,Target,Tweet,Stance,Opinion towards
0,worker_1,2370.0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,AGAINST,TARGET
1,worker_2,2370.0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,FAVOR,TARGET
2,worker_3,2370.0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,AGAINST,OTHER
3,worker_4,2370.0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,AGAINST,OTHER
4,worker_5,2370.0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,AGAINST,TARGET
...,...,...,...,...,...,...
53094,worker_113,10951.0,Hillary Clinton,@TheAtlantic cause #Hillary Clinton is a LYIN...,AGAINST,TARGET
53095,worker_85,10951.0,Hillary Clinton,@TheAtlantic cause #Hillary Clinton is a LYIN...,AGAINST,OTHER
53096,worker_56,10951.0,Hillary Clinton,@TheAtlantic cause #Hillary Clinton is a LYIN...,AGAINST,TARGET
53097,worker_84,10951.0,Hillary Clinton,@TheAtlantic cause #Hillary Clinton is a LYIN...,AGAINST,TARGET


In [5]:
unique_labels = df['Stance'].unique()
print(unique_labels)

['AGAINST' 'FAVOR' 'NONE' 'NEUTRAL']


In [6]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['Stance'] = encoder.fit_transform(df['Stance'])

In [7]:
unique_labels = df['Stance'].unique()
print(unique_labels)

[0 1 3 2]


In [8]:
df_majority = df[df['Stance'] == 0]
df_minority = df[df['Stance'] == 1]
df_minority1 = df[df['Stance'] == 2]
df_minority2 = df[df['Stance'] == 3]
print(len(df_majority))
print(len(df_minority))
print(len(df_minority1))
print(len(df_minority2))


25515
11546
2465
13573


In [9]:
def data_preprocessing(df):
  from sklearn.utils import resample
  # Separate the majority and minority classes

  #here we take 'AGAINST' 'FAVOR' 'NONE' stances we delete NEUTRAL since it have less no.of tweets i.e.,2465
  df_majority_1 = df[df['Stance'] == 0]
  df_majority_2 = df[df['Stance'] == 3]
  df_majority_3 = df[df['Stance'] == 1]
  df_minority=df_majority_1 
  l=[df_majority_2,df_majority_3]
  for i in range(2):
    if(len(df_minority)>len(l[i])):
      df_minority=l[i]
  # Resample the majority class to match the number of samples in the minority class
  df_majority_1_resampled = resample(df_majority_1, replace=False, n_samples=len(df_minority), random_state=42)
  df_majority_2_resampled = resample(df_majority_2, replace=False, n_samples=len(df_minority), random_state=42)
  df_majority_3_resampled = resample(df_majority_3, replace=False, n_samples=len(df_minority), random_state=42)
  
  # Concatenate the resampled majority class with the minority class
  df_resampled = pd.concat([df_majority_1, df_majority_2, df_majority_3])



  # Shuffle the rows of the resampled dataset
  df_resampled = df_resampled.sample(frac=1, random_state=42)
  df = df_resampled
  def clean_data(review):
      review = re.sub('[^a-zA-Z]', ' ', review)
      review = review.lower()
      return review

  df['Tweet'] = df['Tweet'].apply(clean_data)

  def remove_stop_words(review):
      review_minus_sw = []
      stop_words = stopwords.words('english')
      review = review.split()
      review = [review_minus_sw.append(word) for word in review if word not in stop_words]
      review = ' '.join(review_minus_sw)
      return review

  df['Tweet'] = df['Tweet'].apply(remove_stop_words)

  def lematize(review):
      lemmatizer = WordNetLemmatizer()
      review = review.split()
      review = [lemmatizer.lemmatize(word) for word in review]
      review = ' '.join(review)
      return review
  df['Tweet'] = df['Tweet'].apply(lematize)

  message = df['Tweet']
  stance = df['Stance'] 
  #splitting train and test data
  x_train, x_test , y_train, y_test = train_test_split(message, stance , test_size = 0.20)

  # model = Word2Vec(words, size=100, window=5, min_count=1, workers=4)
  vocab_size = 10000
  X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in x_train]
  X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in x_test]
 # X_train, X_test , y_train, y_test = train_test_split(input_data, stance, test_size=0.20)

  #apply padding for each sentence have same size 
  max_length = 100
  X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
  X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

  

  def define_rnn_model():
     model = Sequential()
     model.add(Embedding(vocab_size, 100, input_length=max_length))
     model.add(LSTM(128))
     model.add(Dense(1, activation='sigmoid'))
     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
     return model

  # call the define_rnn_model function to create the model object
  model = define_rnn_model()
  # print the model summary
  model.summary()
  #calling adam optimizer
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
   
  #visualizing each eppoch
  history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
  #caluclating loss and accuracy
  loss, accuracy = model.evaluate(X_test,y_test)

  #printing accuracy
  print('Testing Accuracy is {} '.format(accuracy*100))
    
  y_prob = model.predict(X_test)
  y_predict = (y_prob>=0.5).astype(int)
  from sklearn.metrics import classification_report
  print(classification_report(y_test,y_predict,target_names = ["Against","Favor","None"]))




In [10]:
for i in df["Target"].unique():
  tar = df[df["Target"]==i]
  print("*******************",i,"*********************")
  data_preprocessing(tar)


******************* Legalization of Abortion *********************
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,117,377
Trainable params: 1,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
278/278 [==============================] - 76s 260ms/step - loss: -0.1496 - acc: 0.1919 - val_loss: 0.2276 - val_acc: 0.2005
Epoch 2/10
278/278 [==============================] - 73s 263ms/step - loss: -0.3294 - acc: 0.1910 - val_lo

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     Against       0.00      0.00      0.00      1197
       Favor       0.20      1.00      0.33       446
        None       0.00      0.00      0.00       581

    accuracy                           0.20      2224
   macro avg       0.07      0.33      0.11      2224
weighted avg       0.04      0.20      0.07      2224

******************* Hillary Clinton *********************
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total param

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     Against       0.00      0.00      0.00      1189
       Favor       0.20      1.00      0.34       454
        None       0.00      0.00      0.00       603

    accuracy                           0.20      2246
   macro avg       0.07      0.33      0.11      2246
weighted avg       0.04      0.20      0.07      2246

******************* Feminist Movement *********************
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 lstm_2 (LSTM)               (None, 128)               117248    
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total par

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 lstm_3 (LSTM)               (None, 128)               117248    
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,117,377
Trainable params: 1,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
221/221 [==============================] - 59s 252ms/step - loss: 0.4686 - acc: 0.1684 - val_loss: 0.4055 - val_acc: 0.1596
Epoch 2/10
221/221 [==============================] - 54s 246ms/step - loss: 0.4443 - acc: 0.1684 - val_loss: 0.4006 - val_acc: 0.1596
Epoch 3/10
221/221 [==================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 lstm_4 (LSTM)               (None, 128)               117248    
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,117,377
Trainable params: 1,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
211/211 [==============================] - 59s 265ms/step - loss: -5.8853 - acc: 0.2543 - val_loss: -9.5761 - val_acc: 0.2395
Epoch 2/10
211/211 [==============================] - 52s 248ms/step - loss: -11.9261 - acc: 0.2543 - val_loss: -15.4327 - val_acc: 0.2395
Epoch 3/10
211/211 [============

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
data_preprocessing(df)